In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import math
import plotly.graph_objects as go
import plotly.express as px
from tqdm import tqdm
from functions import *
from tickers_list import *
sns.set_style('whitegrid')

In [2]:
## HARDCODE VALUES

stocks = nifty50_tickers

INITIAL_AMT_INVESTED = 10000
NUM_OF_SIMULATIONS = 1000000
start_date='2013-01-01'
end_date='2023-01-01'

In [3]:
def fetch_data(stock_list, start_date, end_date, div=True):
    
    """Returns stock_price_df, stock_dividend_df"""
    
    if type(stock_list)!=list:
        stock_list = [stock_list]
    
    for i in range(len(stock_list)):
        stock_list[i] = stock_list[i].upper()

    stock_price = {}
    stock_dividend = {}
    for i in tqdm(range(len(stock_list))):
        ticker = yf.Ticker(stock_list[i])
        stock_data = ticker.history(start=start_date, end=end_date)
        stock_price[stock_list[i]] = stock_data['Close']
        if div:
            stock_dividend[stock_list[i]] = stock_data['Dividends']
    if div:    
        return pd.DataFrame(stock_price), pd.DataFrame(stock_dividend)
    else:
        return pd.DataFrame(stock_price)

In [4]:
## GATHER STOCK PRICES

stock_prices = fetch_data(stocks, start_date=start_date, end_date=end_date, div=False)
stock_prices = stock_prices.dropna(axis=1)

## CALCULATE LOG RETURNS

log_returns = np.log(stock_prices/stock_prices.shift(1))[1:]

## DOWNSIDE DEVIATION (SORTINO RATIO)

log_returns_down_only = log_returns.applymap(lambda x: 0 if x>0 else x)

 35%|████████████████████████████▍                                                     | 17/49 [00:15<00:25,  1.25it/s]

HDFC.NS: No data found for this date range, symbol may be delisted


100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [00:40<00:00,  1.22it/s]


In [5]:
## temp cell
stock_prices.shape

(248, 48)

In [81]:
def calculate_beta(log_return_portfolio, log_return_market):
    
    intersection_index = np.intersect1d(log_return_portfolio.index, log_return_market.index)
    
    log_return_market = log_return_market.loc[intersection_index,log_return_market.columns[0]]
    log_return_portfolio = log_return_portfolio[intersection_index]
    
    covariance = np.cov(log_return_portfolio, log_return_market)[0,1]
    variance_market = log_return_market.var()
    
    beta = covariance/variance_market
    
    if type(beta) == np.float64:
        return beta
    else:
        return beta[0]

In [82]:
portfolio_weights = []
portfolio_num_of_shares = []
portfolio_returns = []
portfolio_risks = []
portfolio_risks_sortino = []
portfolio_value = []
portfolio_beta = []
NUM_TRADING_DAYS = len(stock_prices)

nifty50_price,temp = fetch_data("^NSEI", start_date=start_date, end_date=end_date)
log_returns_market = np.log(nifty50_price/nifty50_price.shift(1))[1:]

## MONTE CARLO SIMULATION

for i in tqdm(range(NUM_OF_SIMULATIONS)):
    ## STEP-I: generating random weights
    w = np.random.rand(log_returns.shape[1])
    w = np.expand_dims(w, axis=0)
    w = w/np.sum(w)
    portfolio_weights.append(w.flatten())

    ## STEP-II: calculating returns using average annual returns
    p_returns = (log_returns.mean() @ w.T*NUM_TRADING_DAYS)[0]
    portfolio_returns.append(p_returns)

    ## STEP-III: calculating risks
    p_risks = np.sqrt(w @ log_returns.cov()*NUM_TRADING_DAYS @ w.T)[0][0]
    portfolio_risks.append(p_risks)
    
    p_risks = np.sqrt(w @ log_returns_down_only.cov()*NUM_TRADING_DAYS @ w.T)[0][0]
    portfolio_risks_sortino.append(p_risks)

    ## CALCULATING PORTFOLIO VALUE
    num_of_shares = INITIAL_AMT_INVESTED*w[0]/stock_prices.iloc[0,:]
    num_of_shares = np.expand_dims(num_of_shares, axis=0)
    portfolio_num_of_shares.append(num_of_shares)

    portfolio_value_i = np.sum(num_of_shares*stock_prices, axis=1)
    ## portfolio_value.append(portfolio_value_i) NOT STORING PORTOFOLIO VALUES SINCE THE NUM OF SIMULATIONS IS TOO LARGE

    log_returns_portfolio_value_i = np.log(portfolio_value_i/portfolio_value_i.shift(1))[1:]

    ## STEP-IV: calculating portfolio beta
    beta = calculate_beta(log_returns_portfolio_value_i, log_returns_market)
    portfolio_beta.append(beta)

  0%|▏                                                                       | 2849/1000000 [03:01<17:36:25, 15.73it/s]


KeyboardInterrupt: 

In [5]:
portfolio_weights = np.array(portfolio_weights)
portfolio_num_of_shares = np.array(portfolio_num_of_shares)
portfolio_returns = np.array(portfolio_returns)
portfolio_risks = np.array(portfolio_risks)
portfolio_risks_sortino = np.array(portfolio_risks_sortino)
portfolio_value = np.array(portfolio_value)
portfolio_beta = np.array(portfolio_beta)



# taking IR of government issued 10-year bonds as risk-free rate
risk_free_rate = 0.07

## CALCULATING RATIOS

sharpe_ratios = (portfolio_returns - risk_free_rate) / portfolio_risks
sortino_ratios = (portfolio_returns - risk_free_rate) / portfolio_risks_sortino

## TREYNOR RATIO
treynor_ratios = (portfolio_returns - risk_free_rate) / portfolio_beta

## JENSONS ALPHA
CAPM_expected_return = risk_free_rate + beta*(np.sum(log_returns_market) - risk_free_rate)
CAPM_expected_return = CAPM_expected_return[0]
jenson_alpha = portfolio_returns - CAPM_expected_return

In [6]:
## CREATE A TABLE FOR ALL THE SIMULATIONS

df_sims = pd.DataFrame({'Portfolio_return':portfolio_returns,'Portfolio_risk':portfolio_risks,
                        'Sharpe':sharpe_ratios,'Sortino':sortino_ratios,'Treynor':treynor_ratios,'JensonAlpha':jenson_alpha})
df_sims = pd.concat([df_sims,weights_to_dataframe(portfolio_weights, stocks)], axis=1)

In [7]:
df_sims

,Portfolio_return,Portfolio_risk,Sharpe,Sortino,Treynor,JensonAlpha,LUPIN.NS,VOLTAS.NS,YESBANK.NS
0,0.203210,0.179947,0.740276,1.234616,0.216796,0.085646,0.234575,0.328981,0.436444
1,0.078110,0.249829,0.032462,0.054358,0.011211,-0.039455,0.083723,0.171656,0.744621
2,0.346813,0.167596,1.651671,2.816794,0.469079,0.229249,0.493082,0.266914,0.240004
3,0.154796,0.198314,0.427583,0.712296,0.132086,0.037231,0.164039,0.302655,0.533306
4,0.235720,0.173715,0.953975,1.597229,0.272652,0.118155,0.293117,0.314873,0.392010
...,...,...,...,...,...,...,...,...,...
4995,0.217676,0.195001,0.757310,1.275751,0.225587,0.100112,0.305812,0.194224,0.499964
4996,0.256828,0.165987,1.125556,1.884421,0.317640,0.139263,0.314246,0.353712,0.332042
4997,0.368799,0.162717,1.836306,3.114221,0.566728,0.251234,0.465669,0.447886,0.086445
4998,0.297489,0.162276,1.401860,2.361844,0.392962,0.179924,0.384573,0.344295,0.271131


In [8]:
sharpe_max_indices = np.argsort(sharpe_ratios)[-1:-11:-1]
sortino_max_indices = np.argsort(sortino_ratios)[-1:-11:-1]
treynor_max_indices = np.argsort(treynor_ratios)[-1:-11:-1]
jenson_alpha_max_indices = np.argsort(jenson_alpha)[-1:-11:-1]

In [9]:
df_sims.iloc[sharpe_max_indices,:]

,Portfolio_return,Portfolio_risk,Sharpe,Sortino,Treynor,JensonAlpha,LUPIN.NS,VOLTAS.NS,YESBANK.NS
347,0.552761,0.219054,2.203847,3.847009,0.815654,0.435197,0.886673,0.112928,0.000399
2521,0.524094,0.206250,2.201666,3.832980,0.785519,0.406529,0.817690,0.174723,0.007586
4203,0.508552,0.199582,2.197356,3.817747,0.770828,0.390987,0.778294,0.213907,0.007799
2919,0.508807,0.199757,2.196708,3.817024,0.770355,0.391242,0.779535,0.211573,0.008892
1520,0.508904,0.199814,2.196559,3.816882,0.770269,0.391339,0.779930,0.210908,0.009163
2342,0.554866,0.220869,2.195264,3.834233,0.812447,0.437301,0.896031,0.096183,0.007786
3137,0.501681,0.196780,2.193717,3.807598,0.764331,0.384116,0.760717,0.231684,0.007599
2970,0.573743,0.229776,2.192316,3.833648,0.832583,0.456178,0.940323,0.058707,0.000971
2382,0.494949,0.193958,2.190927,3.797731,0.760598,0.377384,0.741212,0.255596,0.003192
4404,0.547597,0.218004,2.190770,3.825216,0.801621,0.430032,0.881322,0.103939,0.014738


In [10]:
df_sims.iloc[treynor_max_indices,:]

,Portfolio_return,Portfolio_risk,Sharpe,Sortino,Treynor,JensonAlpha,LUPIN.NS,VOLTAS.NS,YESBANK.NS
2970,0.573743,0.229776,2.192316,3.833648,0.832583,0.456178,0.940323,0.058707,0.000971
347,0.552761,0.219054,2.203847,3.847009,0.815654,0.435197,0.886673,0.112928,0.000399
3547,0.561547,0.224829,2.186315,3.821706,0.814715,0.443983,0.915688,0.071601,0.012711
291,0.568360,0.229700,2.169608,3.796279,0.813464,0.450795,0.938549,0.038525,0.022926
3294,0.563488,0.226462,2.179121,3.810644,0.812900,0.445924,0.923392,0.058793,0.017815
2342,0.554866,0.220869,2.195264,3.834233,0.812447,0.437301,0.896031,0.096183,0.007786
4347,0.556936,0.223387,2.179788,3.810281,0.805733,0.439372,0.908163,0.071390,0.020447
4404,0.547597,0.218004,2.190770,3.825216,0.801621,0.430032,0.881322,0.103939,0.014738
4086,0.545122,0.217127,2.188225,3.820507,0.797391,0.427558,0.876751,0.105339,0.017909
3915,0.558766,0.227188,2.151375,3.764856,0.793292,0.441202,0.924389,0.033830,0.041781


In [11]:
df_sims.iloc[sortino_max_indices,:]

,Portfolio_return,Portfolio_risk,Sharpe,Sortino,Treynor,JensonAlpha,LUPIN.NS,VOLTAS.NS,YESBANK.NS
347,0.552761,0.219054,2.203847,3.847009,0.815654,0.435197,0.886673,0.112928,0.000399
2342,0.554866,0.220869,2.195264,3.834233,0.812447,0.437301,0.896031,0.096183,0.007786
2970,0.573743,0.229776,2.192316,3.833648,0.832583,0.456178,0.940323,0.058707,0.000971
2521,0.524094,0.206250,2.201666,3.832980,0.785519,0.406529,0.817690,0.174723,0.007586
4404,0.547597,0.218004,2.190770,3.825216,0.801621,0.430032,0.881322,0.103939,0.014738
3547,0.561547,0.224829,2.186315,3.821706,0.814715,0.443983,0.915688,0.071601,0.012711
4086,0.545122,0.217127,2.188225,3.820507,0.797391,0.427558,0.876751,0.105339,0.017909
2917,0.530199,0.210137,2.189994,3.818108,0.782953,0.412635,0.839891,0.140212,0.019897
4203,0.508552,0.199582,2.197356,3.817747,0.770828,0.390987,0.778294,0.213907,0.007799
2919,0.508807,0.199757,2.196708,3.817024,0.770355,0.391242,0.779535,0.211573,0.008892


In [12]:
df_sims.iloc[jenson_alpha_max_indices,:]

,Portfolio_return,Portfolio_risk,Sharpe,Sortino,Treynor,JensonAlpha,LUPIN.NS,VOLTAS.NS,YESBANK.NS
2970,0.573743,0.229776,2.192316,3.833648,0.832583,0.456178,0.940323,0.058707,0.000971
291,0.568360,0.229700,2.169608,3.796279,0.813464,0.450795,0.938549,0.038525,0.022926
3294,0.563488,0.226462,2.179121,3.810644,0.812900,0.445924,0.923392,0.058793,0.017815
3547,0.561547,0.224829,2.186315,3.821706,0.814715,0.443983,0.915688,0.071601,0.012711
1768,0.559154,0.229666,2.129845,3.729659,0.783941,0.441589,0.933565,0.009553,0.056882
3915,0.558766,0.227188,2.151375,3.764856,0.793292,0.441202,0.924389,0.033830,0.041781
4347,0.556936,0.223387,2.179788,3.810281,0.805733,0.439372,0.908163,0.071390,0.020447
2342,0.554866,0.220869,2.195264,3.834233,0.812447,0.437301,0.896031,0.096183,0.007786
347,0.552761,0.219054,2.203847,3.847009,0.815654,0.435197,0.886673,0.112928,0.000399
994,0.551577,0.225708,2.133631,3.734347,0.777083,0.434012,0.914818,0.027352,0.057830


In [71]:
log_return_market2 = log_returns_market
log_return_portfolio = log_returns_portfolio_value_i

intersection_index = np.intersect1d(log_return_portfolio.index, log_return_market2.index)
log_return_market2 = log_return_market2.loc[intersection_index,log_return_market2.columns[0]]
log_return_portfolio = log_return_portfolio[intersection_index]

covariance = np.cov(log_return_portfolio, log_return_market2)[0,1]
variance_market = log_return_market2.var()
    
beta = covariance/variance_market
type(beta)

numpy.float64